In [31]:
from dotenv import load_dotenv

load_dotenv()

True

In [32]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [51]:
import pandas as pd

df = pd.read_csv('./faker/dataset_3.csv', dtype={'Patient Phone': str})
# Change date to YYYYMMDD-interger 
df['Visit Date'] = pd.to_datetime(df['Visit Date']).dt.strftime('%Y%m%d').astype(int)
df

,Visit ID,Description,Visit Date,Patient ID,Diagnosis Category,Diagnosis Sub Category,Treatment Category,Treatment Sub Category,New Patient,Consulting Physician,...,Patient Gender,Patient Age,Patient Age Range,Patient Blood Type,Patient Insurance Number,Patient Phone,Patient Address,Patient Occupation,Patient Emergency Contact,Intended Purposes
0,VQC513203,The patient presented with chronic lower back ...,20170101,PZM508653,Chronic Pain,Vertebral Disc Problem,Pharmacy/Prescription Drugs,Non-FDA Approved Use,True,Dr. Jerry Daniels,...,Female,43,41-50,O-,G264037622,017588215469,Pärtzeltweg 2\n22301 Neunburg vorm Wald,"Surveyor, quantity","Jennifer Bailey, 015680180768.","['Care', 'Research', 'Insurance', 'Support', '..."
1,VVC435406,The patient presented with symptoms of fatigue...,20170101,PSN036517,Endocrine/ Metabolic,Hormone Deficiency,Pharmacy/Prescription Drugs,Hormones,True,Dr. Michelle Lamb,...,Female,36,31-40,O+,Y133547589,015087781378,Kira-Gorlitz-Allee 8\n67100 Rosenheim,Copy,"Dennis Carlson, 015182104709.","['Care', 'Research', 'Insurance', 'Support', '..."
2,VKT437745,The patient presented with delayed speech deve...,20170101,PPD253419,Pediatrics,Delayed Speech,Rehabilitation Services - Outpatient,Speech Therapy,True,Dr. Michelle Lamb,...,Male,5,0-10,O+,F115599209,016368315207,Thiesstr. 3/5\n68745 Bremen,Claims inspector/assessor,"Veronica Harris, 015034485673.","['Care', 'Research', 'Insurance', 'Support', '..."
3,VJG208744,The patient presented with symptoms of muscle ...,20170101,PBN488954,Central Nervous System/ Neuromuscular,NaN,Pharmacy/Prescription Drugs,Non-FDA Approved Use,True,Dr. James Barber,...,Male,52,51-64,AB+,W720918648,016217695326,Schmidtkeallee 53\n54913 Siegen,Tourism officer,"Michelle Graham, 017438105819.","['Care', 'Research', 'Insurance', 'Support', '..."
4,VAF235393,The patient presented with a diagnosis of brea...,20170101,PJG173047,Cancer,Breast Cancer,Cancer Treatment,Surgery,True,Dr. James Barber,...,Female,36,31-40,A+,Q198981012,017645195473,Ida-Fliegner-Ring 7/6\n53118 Wolgast,Engineering geologist,"Peter Stout, 015017527431.","['Care', 'Research', 'Insurance', 'Support', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11684,VTL998110,"Today, I met with a patient who was diagnosed ...",20181230,PND058072,Mental,Depression,Electrical/ Thermal/ Radiofreq. Interventions,Transcranial Magnetic Stimulation,False,Dr. Alexandria Gaines,...,Male,51,51-64,A-,Private,016976645290,Wolfram-auch Schlauchin-Ring 3/3\n94816 Neuruppin,"Research officer, political party","Emily Garrett, 016593887796.","['Care', 'Research', 'Insurance', 'Support', '..."
11685,VLH032189,The patient was diagnosed with liver cancer af...,20181230,PPC895292,Cancer,Liver Cancer,Special Procedure,NaN,False,Dr. Eddie Young,...,Male,62,51-64,B-,Q100759966,016394851011,Sölzerring 05\n21768 Altötting,"Engineer, petroleum","Linda Flores, 016973980632.","['Care', 'Insurance', 'Support', 'Public', 'Tr..."
11686,VQZ422704,Today I saw a patient with a chromosomal anoma...,20181230,PFS771409,Genetic,Chromosomal Anomalies,Pharmacy/Prescription Drugs,Hormones,False,Dr. James Barber,...,Female,7,0-10,A+,L605331052,016453400773,Mielcarekplatz 1/9\n83357 Melle,Scientific laboratory technician,"Michael Weiss, 017716313028.","['Care', 'Research', 'Insurance', 'Support', '..."
11687,VUT814433,The patient presented with symptoms of persist...,20181230,PVV930449,Mental,NaN,NaN,NaN,True,Dr. Eddie Young,...,Female,16,10-20,A+,G215159968,017155908480,Dussen vangasse 501\n61560 Aachen,Data processing manager,"Tammy Whitney, 017700151177.","['Care', 'Research', 'Insurance', 'Support', '..."


In [52]:
from langchain_core.documents import Document

def generate_db_docs(df, page_content_column, limit = None):
    if limit is not None:
        df = df.iloc[:limit]

    # Ensure the specified page_content_column exists in the DataFrame
    if page_content_column not in df.columns:
        raise ValueError(f"{page_content_column} does not exist in the DataFrame.")

    docs = []
    cols = list(df.columns)

    for row in df.itertuples(index=False, name=None):
        # Dynamically get the page content using the column name
        page_content_index = cols.index(page_content_column)
        page_content = row[page_content_index]

        # Construct metadata dictionary from the rest of the columns
        metadata = {col: row[cols.index(col)] for col in cols if col != page_content_column}

        # Create Document object and append to docs list
        docs.append(Document(page_content=page_content, metadata=metadata))

    return docs

docs = generate_db_docs(df, 'Description', 100)


In [53]:
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="Visit ID",
        description="Unique ID for patient visting a medical center.",
        type="string",
    ),
    AttributeInfo(
        name="Visit Date",
        description="The date the visit to the medical center took place. Format: YYYYMMDD.",
        type="integer",
    ),
    AttributeInfo(
        name="Patient ID",
        description="The ID corresponding to a patient. A patient might visit multiple times, therefore the ID is not unique.",
        type="string",
    ),
    AttributeInfo(
        name="Diagnosis Category", 
        description="Category for medical diagnosis.", 
        type="string"
    ),
    AttributeInfo(
        name="Diagnosis Sub Category", 
        description="Sub category for medical diagnosis.", 
        type="string"
    ),
    AttributeInfo(
        name="Treatment Category", 
        description="Category for medical treatment based on diganosis.", 
        type="string"
    ),
    AttributeInfo(
        name="Treatment Sub Category", 
        description="Sub category for medical treatment based on diganosis.", 
        type="string"
    ),
    AttributeInfo(
        name="New Patient", 
        description="A flag indicating if it is the first time a patient is visiting the medical center. 'True' if it is the first time, 'False' if not.", 
        type="boolean"
    ),
    AttributeInfo(
        name="Consulting Physician", 
        description="The name of the consulting physician. Availbale options: 'Dr. Jerry Daniels', 'Dr. Michelle Lamb', 'Dr. James Barber', 'Dr. Shelly Hunt', 'Dr. Alexandria Gaines', 'Dr. Eddie Young'.", 
        type="string"
    ),
    AttributeInfo(
        name="Patient Name", 
        description="The name of the patient seeking medical assistance.", 
        type="string"
    ),
    AttributeInfo(
        name="Patient Gender", 
        description="The gender of the patient seeking medical assistance. Available options: 'Female', 'Male', 'Other'.", 
        type="string"
    ),
    AttributeInfo(
        name="Patient Age", 
        description="The age of the patient seeking medical assistance.", 
        type="integer"
    ),
    AttributeInfo(
        name="Patient Age Range", 
        description="The age range of the patient seeking medical assistance. Available options: '0-10', '10-20', '21-30', '31-40', '41-50', '51-64', '65+'.", 
        type="string"
    ),
    AttributeInfo(
        name="Patient Blood Type", 
        description="The blood type of the patient seeking medical assistance. Available options: 'O-', 'O+', 'AB+', 'A+', 'B+', 'A-', 'B-', 'AB-'.", 
        type="string"
    ),
    AttributeInfo(
        name="Patient Insurance Number", 
        description="The insurance number of the patient seeking medical assistance.", 
        type="string"
    ),
    AttributeInfo(
        name="Patient Phone", 
        description="The moblie phone number of the patient seeking medical assistance.", 
        type="string"
    ),
    AttributeInfo(
        name="Patient Address", 
        description="The address of the patient seeking medical assistance.", 
        type="string"
    ),
    AttributeInfo(
        name="Patient Occupation", 
        description="The occupation of the patient seeking medical assistance.", 
        type="string"
    ),
    AttributeInfo(
        name="Patient Emergency Contact", 
        description="The emergency contact of the patient seeking medical assistance. The contact includes a name and an address of the emergency contact.", 
        type="string"
    ),
    AttributeInfo(
        name="Intended Purposes", 
        description="A list of intended access purposes for the data generated at the visit to the medical center. The list can contain a set of the following options: 'Care', 'Research', 'Insurance', 'Support', 'Public', 'Trial', 'Product', 'Marketing'.", 
        type="list[string]"
    ),
]


In [54]:
import os

persist_directory = "./chroma_medical_db"

# Check if the persistence directory exists
if not os.path.exists(persist_directory):
    # If not, create using docs
    vectorstore = Chroma.from_documents(docs, OpenAIEmbeddings(), persist_directory=persist_directory)
    print('Creating vectorstore...')
else:
    # If the directory exists, initialize Chroma to use the existing data
    vectorstore = Chroma(persist_directory=persist_directory, embedding_function=OpenAIEmbeddings())
    print('Using existing vectorstore...')

Creating vectorstore...


In [64]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

document_content_description = "A summary of of a visit at a medical center for the docters perspective. The text includes symptoms dignosis and proposed treatment."
llm = ChatOpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    verbose=True,
    #return_intermediate_steps=True,
    #enable_limit=True,
)

In [65]:
response = retriever.invoke("Get me visits with diganosis of cancer.")
response

[Document(page_content='The patient presented with a suspicious mole that was later diagnosed as skin cancer. The treatment plan involved initiating chemotherapy to target and eradicate the cancerous cells. The patient was informed about the potential side effects of chemotherapy and the importance of adhering to the treatment schedule for optimal outcomes. Follow-up appointments were scheduled to monitor the response to the chemotherapy and adjust the treatment plan as needed.', metadata={'Consulting Physician': 'Dr. Alexandria Gaines', 'Diagnosis Category': 'Cancer', 'Diagnosis Sub Category': 'Skin Cancer', 'Intended Purposes': "['Care', 'Insurance', 'Support', 'Public', 'Trial']", 'New Patient': True, 'Patient Address': 'Oestrovskyallee 2\n51854 Hildburghausen', 'Patient Age': 59, 'Patient Age Range': '51-64', 'Patient Blood Type': 'O+', 'Patient Emergency Contact': 'Barbara Thompson, 017647870088.', 'Patient Gender': 'Male', 'Patient ID': 'PGD755392', 'Patient Insurance Number': 'V

In [60]:
response = retriever.invoke("Get me patients for that allow data access for research.")
response

[Document(page_content='The patient has been diagnosed with breast cancer. They are currently undergoing investigational treatment as part of their cancer treatment plan. The goal is to explore new and innovative approaches to managing their condition and improve their chances of successful outcomes. Close monitoring and regular assessments will be conducted to track the effectiveness of this investigational treatment.', metadata={'Consulting Physician': 'Dr. James Barber', 'Diagnosis Category': 'Cancer', 'Diagnosis Sub Category': 'Breast Cancer', 'Intended Purposes': "['Care', 'Research', 'Insurance', 'Support', 'Product']", 'New Patient': True, 'Patient Address': 'Henny-Bohlander-Gasse 536\n17817 Hagenow', 'Patient Age': 54, 'Patient Age Range': '51-64', 'Patient Blood Type': 'AB+', 'Patient Emergency Contact': 'Michelle Rivera, 015169080560.', 'Patient Gender': 'Female', 'Patient ID': 'PDM236997', 'Patient Insurance Number': 'P146996041', 'Patient Name': 'Rhonda Garcia', 'Patient Oc

In [62]:
response = retriever.invoke("Get me patients with blood type A-")
response

[Document(page_content="The patient presented with elevated blood pressure readings consistent with a diagnosis of hypertension. As part of the treatment plan, I discussed the option of prescribing formulary versus non-formulary medications to help manage their condition. We reviewed the benefits and potential side effects of each type of medication to determine the most suitable choice for the patient's specific needs.", metadata={'Consulting Physician': 'Dr. Jerry Daniels', 'Diagnosis Category': 'Cardiac/Circulatory', 'Diagnosis Sub Category': 'Hypertension', 'Intended Purposes': "['Care', 'Research', 'Insurance', 'Support', 'Public', 'Trial']", 'New Patient': True, 'Patient Address': 'Minna-Klotz-Gasse 1\n55843 Hannoversch Münden', 'Patient Age': 61, 'Patient Age Range': '51-64', 'Patient Blood Type': 'A-', 'Patient Emergency Contact': 'Paul Vincent, 017740391054.', 'Patient Gender': 'Male', 'Patient ID': 'PVW942378', 'Patient Insurance Number': 'Y945492970', 'Patient Name': 'Larry 

In [63]:
response = retriever.invoke("Get me patients that struggle with blood pressure.")
response

[Document(page_content="The patient presented with elevated blood pressure readings consistent with a diagnosis of hypertension. As part of the treatment plan, I discussed the option of prescribing formulary versus non-formulary medications to help manage their condition. We reviewed the benefits and potential side effects of each type of medication to determine the most suitable choice for the patient's specific needs.", metadata={'Consulting Physician': 'Dr. Jerry Daniels', 'Diagnosis Category': 'Cardiac/Circulatory', 'Diagnosis Sub Category': 'Hypertension', 'Intended Purposes': "['Care', 'Research', 'Insurance', 'Support', 'Public', 'Trial']", 'New Patient': True, 'Patient Address': 'Minna-Klotz-Gasse 1\n55843 Hannoversch Münden', 'Patient Age': 61, 'Patient Age Range': '51-64', 'Patient Blood Type': 'A-', 'Patient Emergency Contact': 'Paul Vincent, 017740391054.', 'Patient Gender': 'Male', 'Patient ID': 'PVW942378', 'Patient Insurance Number': 'Y945492970', 'Patient Name': 'Larry 

In [76]:
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)

prompt = get_query_constructor_prompt(
    document_content_description,
    metadata_field_info,
)
output_parser = StructuredQueryOutputParser.from_components()
query_constructor = prompt | llm | output_parser

In [ ]:
from langchain.retrievers.self_query.chroma import ChromaTranslator

retriever = SelfQueryRetriever(
    query_constructor=query_constructor,
    vectorstore=vectorstore,
    structured_query_translator=ChromaTranslator(),
)

In [83]:
print(query_constructor.invoke(
        {
            "query": "Get all patients with cancer."
        }
    )
)

print(retriever.invoke(
        "Get all patients with cancer."
    )
)

query='cancer' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='Diagnosis Category', value='Cancer') limit=None
[Document(page_content='The patient presented with a suspicious mole that was later diagnosed as skin cancer. The treatment plan involved initiating chemotherapy to target and eradicate the cancerous cells. The patient was informed about the potential side effects of chemotherapy and the importance of adhering to the treatment schedule for optimal outcomes. Follow-up appointments were scheduled to monitor the response to the chemotherapy and adjust the treatment plan as needed.', metadata={'Consulting Physician': 'Dr. Alexandria Gaines', 'Diagnosis Category': 'Cancer', 'Diagnosis Sub Category': 'Skin Cancer', 'Intended Purposes': "['Care', 'Insurance', 'Support', 'Public', 'Trial']", 'New Patient': True, 'Patient Address': 'Oestrovskyallee 2\n51854 Hildburghausen', 'Patient Age': 59, 'Patient Age Range': '51-64', 'Patient Blood Type': 'O+', 'Patient Emergency Co

In [81]:
retriever.invoke(
    "Get all patients with cancer."
)

[Document(page_content='The patient presented with a suspicious mole that was later diagnosed as skin cancer. The treatment plan involved initiating chemotherapy to target and eradicate the cancerous cells. The patient was informed about the potential side effects of chemotherapy and the importance of adhering to the treatment schedule for optimal outcomes. Follow-up appointments were scheduled to monitor the response to the chemotherapy and adjust the treatment plan as needed.', metadata={'Consulting Physician': 'Dr. Alexandria Gaines', 'Diagnosis Category': 'Cancer', 'Diagnosis Sub Category': 'Skin Cancer', 'Intended Purposes': "['Care', 'Insurance', 'Support', 'Public', 'Trial']", 'New Patient': True, 'Patient Address': 'Oestrovskyallee 2\n51854 Hildburghausen', 'Patient Age': 59, 'Patient Age Range': '51-64', 'Patient Blood Type': 'O+', 'Patient Emergency Contact': 'Barbara Thompson, 017647870088.', 'Patient Gender': 'Male', 'Patient ID': 'PGD755392', 'Patient Insurance Number': 'V